In [39]:
import numpy as np

%reload_ext autoreload
%autoreload 2

In [40]:
def load_data(sub_sample=True, add_outlier=False):
    """Load data and convert it to the metrics system."""
    path_dataset = "data/height_weight_genders.csv"
    data = np.genfromtxt(
        path_dataset, delimiter=",", skip_header=1, usecols=[1, 2])
    height = data[:, 0]
    weight = data[:, 1]
    gender = np.genfromtxt(
        path_dataset, delimiter=",", skip_header=1, usecols=[0],
        converters={0: lambda x: 0 if b"Male" in x else 1})
    # Convert to metric system
    height *= 0.025
    weight *= 0.454

    # sub-sample
    if sub_sample:
        height = height[::50]
        weight = weight[::50]

    if add_outlier:
        # outlier experiment
        height = np.concatenate([height, [1.1, 1.2]])
        weight = np.concatenate([weight, [51.5/0.454, 55.3/0.454]])

    return height, weight, gender


def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x, mean_x, std_x


def build_model_data(height, weight):
    """Form (y,tX) to get regression data in matrix form."""
    y = weight
    x = height
    num_samples = len(y)
    tx = np.c_[np.ones(num_samples), x]
    return y, tx

In [41]:
height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

## Least squares test

### Normal equations

In [42]:
from implementations import least_squares

least_squares(y, tx)

(15.385887868829403, array([73.293922  , 13.47971243]))

### Ridge regression

In [38]:
from implementations import ridge_regression

lambda_ = 1e-5

ridge_regression(y, tx, lambda_)

(15.3858889795253, array([73.29245615, 13.47944285]))

### Gradient Descent

In [27]:
from implementations import least_squares_GD

initial_w = np.zeros(tx.shape[1])
max_iters = 1000
gamma = 0.1

w, loss = least_squares_GD(y, tx, initial_w, max_iters, gamma)

Gradient Descent(0/999): loss=2792.236712759167, w0=7.329392200210504, w1=1.3479712434989137
Gradient Descent(1/999): loss=2264.635056030004, w0=13.925845180399968, w1=2.561145362647917
Gradient Descent(2/999): loss=1837.2777140793814, w0=19.86265286257048, w1=3.6530020698820245
Gradient Descent(3/999): loss=1491.118267099377, w0=25.205779776523947, w1=4.635673106392713
Gradient Descent(4/999): loss=1210.729115045573, w0=30.014593999082074, w1=5.520077039252335
Gradient Descent(5/999): loss=983.6139018819919, w0=34.34252679938438, w1=6.316040578825998
Gradient Descent(6/999): loss=799.6505792194912, w0=38.23766631965647, w1=7.032407764442294
Gradient Descent(7/999): loss=650.6402878628652, w0=41.743291887901336, w1=7.67713823149696
Gradient Descent(8/999): loss=529.9419518639986, w0=44.89835489932172, w1=8.257395651846164
Gradient Descent(9/999): loss=432.1762997049167, w0=47.73791160960007, w1=8.779627330160444
Gradient Descent(10/999): loss=352.98612145606006, w0=50.29351264885058, w

In [7]:
w, loss

(15.385887868829398, array([73.293922  , 13.47971243]))

### SGD

In [29]:
from implementations import least_squares_SGD

max_iters = 10000
gamma = 0.01

w, loss = least_squares_SGD(y, tx, initial_w, max_iters, gamma)

Stochastic Gradient Descent(0/9999): loss=2792.236712759167, w0=0.6516310653751489, w1=-0.18443959308841526
Stochastic Gradient Descent(1/9999): loss=2747.1916294603006, w0=1.4373588736827156, w1=0.4837388094638244
Stochastic Gradient Descent(2/9999): loss=2681.516385420984, w0=2.1427406579950956, w1=0.6698163155867143
Stochastic Gradient Descent(3/9999): loss=2628.677910494988, w0=2.8146776363934016, w1=0.18647948718365592
Stochastic Gradient Descent(4/9999): loss=2587.402852151996, w0=3.7095392014066566, w1=1.4711124486375329
Stochastic Gradient Descent(5/9999): loss=2508.482289562004, w0=4.612513514642081, w1=2.862896212110748
Stochastic Gradient Descent(6/9999): loss=2430.3122171348987, w0=5.151999418043521, w1=2.2608476006059224
Stochastic Gradient Descent(7/9999): loss=2399.9781586810345, w0=5.773651687233181, w1=1.7226048565532959
Stochastic Gradient Descent(8/9999): loss=2363.9941288709774, w0=6.6243373343359435, w1=2.4763980171891773
Stochastic Gradient Descent(9/9999): loss=2

In [30]:
w, loss

(15.677595348164669, array([73.88279844, 13.96858816]))

## Logistic regression test

In [43]:
def load_data(sub_sample=True, add_outlier=False):
    """Load data and convert it to the metric system."""
    path_dataset = "data/height_weight_genders.csv"
    data = np.genfromtxt(
        path_dataset, delimiter=",", skip_header=1, usecols=[1, 2])
    height = data[:, 0]
    weight = data[:, 1]
    gender = np.genfromtxt(
        path_dataset, delimiter=",", skip_header=1, usecols=[0],
        converters={0: lambda x: 0 if b"Male" in x else 1})
    # Convert to metric system
    height *= 0.025
    weight *= 0.454
    return height, weight, gender


def sample_data(y, x, seed, size_samples):
    """sample from dataset."""
    np.random.seed(seed)
    num_observations = y.shape[0]
    random_permuted_indices = np.random.permutation(num_observations)
    y = y[random_permuted_indices]
    x = x[random_permuted_indices]
    return y[:size_samples], x[:size_samples]


def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x


def de_standardize(x, mean_x, std_x):
    """Reverse the procedure of standardization."""
    x = x * std_x
    x = x + mean_x
    return x


def build_model_data(height, weight):
    """Form (y,tX) to get regression data in matrix form."""
    y = weight
    x = height
    num_samples = len(y)
    tx = np.c_[np.ones(num_samples), x]
    return y, tx

In [44]:
# load data.
height, weight, gender = load_data()

# build sampled x and y.
seed = 1
y = np.expand_dims(gender, axis=1)
X = np.c_[height.reshape(-1), weight.reshape(-1)]
y, X = sample_data(y, X, seed, size_samples=200)
x, mean_x, std_x = standardize(X)
tx = np.c_[np.ones((y.shape[0], 1)), x]
initial_w = np.zeros((tx.shape[1], 1))

In [45]:
from implementations import logistic_regression

max_iters = 10000
gamma = 0.01
logistic_regression(y, tx, initial_w, max_iters, gamma)

Current iteration=0, loss=[138.62943611]
||d|| = 105.20575411219552
Current iteration=100, loss=[43.46403231]
||d|| = 2.002274832375109
Current iteration=200, loss=[41.54589281]
||d|| = 0.8954442407385357
Current iteration=300, loss=[41.09863897]
||d|| = 0.47363887489282225
Current iteration=400, loss=[40.96487064]
||d|| = 0.2694193658427241
Current iteration=500, loss=[40.92008946]
||d|| = 0.15904182013823484
Current iteration=600, loss=[40.90418744]
||d|| = 0.09583433339693624
Current iteration=700, loss=[40.89834995]
||d|| = 0.05844053726762646
Current iteration=800, loss=[40.89616497]
||d|| = 0.035892031801379044
Current iteration=900, loss=[40.89533753]
||d|| = 0.02213881348328916
Current iteration=1000, loss=[40.89502196]
||d|| = 0.01369165460565005
Current iteration=1100, loss=[40.89490109]
||d|| = 0.008481294984450714
Current iteration=1200, loss=[40.89485466]
||d|| = 0.005259003912058269
Current iteration=1300, loss=[40.8948368]
||d|| = 0.0032629774424134755
Current iteration=

(array([[ 0.23833206],
        [ 2.46258033],
        [-6.99339341]]), array([40.89482665]))

In [46]:
from implementations import reg_logistic_regression

lambda_ = 1e-5
max_iters = 10000
gamma = 0.01

reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=[138.63243611]
||d|| = 105.20575411219552
Current iteration=100, loss=[43.46706843]
||d|| = 2.0022496228571516
Current iteration=200, loss=[41.54893243]
||d|| = 0.8954183166832301
Current iteration=300, loss=[41.10167156]
||d|| = 0.47361608869549565
Current iteration=400, loss=[40.96789477]
||d|| = 0.2694006788935963
Current iteration=500, loss=[40.92310636]
||d|| = 0.15902714903908877
Current iteration=600, loss=[40.90719887]
||d|| = 0.0958231770339246
Current iteration=700, loss=[40.90135748]
||d|| = 0.05843226404394382
Current iteration=800, loss=[40.89916985]
||d|| = 0.035886021039042926
Current iteration=900, loss=[40.89834066]
||d|| = 0.02213452021573623
Current iteration=1000, loss=[40.89802395]
||d|| = 0.013688631660118616
Current iteration=1100, loss=[40.89790234]
||d|| = 0.008479192123187028
Current iteration=1200, loss=[40.89785545]
||d|| = 0.005257556105314443
Current iteration=1300, loss=[40.8978373]
||d|| = 0.0032619894003265326
Current iteration

(array([[ 0.23832938],
        [ 2.46254347],
        [-6.99332944]]), array([40.89782673]))